### Тема «POS-tagger и NER»

*Задание 1.* Написать теггер на данных с русским языком
* проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
* написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
* сравнить все реализованные методы, сделать выводы  


In [4]:
import pyconll


In [22]:
!wget -O ./dataset/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./dataset/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./dataset/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget -O ./dataset/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu


--2023-04-20 17:36:00--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: './dataset/ru_syntagrus-ud-train-a.conllu'

./dataset/ru_syntag 100%[===================>]  38.85M  5.88MB/s    in 5.7s    

2023-04-20 17:36:08 (6.80 MB/s) - './dataset/ru_syntagrus-ud-train-a.conllu' saved [40736581/40736581]

--2023-04-20 17:36:08--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githu

In [5]:
full_train = pyconll.load_from_file('dataset/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('dataset/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('dataset/ru_syntagrus-ud-train-c.conllu')

# Общая обучающая выборка
full_train.extend([*full_train_b, *full_train_c])


full_test = pyconll.load_from_file('dataset/ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(str(token.form), str(token.upos)) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(str(token.form), str(token.upos)) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([str(token.form) for token in sent])

In [8]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(str(token.form)) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [38]:
all_train_texts = [' '.join(str(token.form) for token in sent) for sent in full_train]
all_test_texts = [' '.join(str(token.form) for token in sent) for sent in full_test]

all_train_labels = [' '.join(str(token.form) for token in sent) for sent in full_train]
all_test_labels = [' '.join(str(token.form) for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [41]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', None),
 ('к', 'ADP'),
 ('ещё', None),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.7101308678950452

In [47]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'PART'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.8782863467673677

In [43]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', None),
 ('вариантах', None),
 ('(', None),
 ('самые', None),
 ('ранние', None),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', None),
 ('лет', None),
 ('старше', None),
 (')', None),
 ('и', 'PART'),
 ('восходит', None),
 ('к', None),
 ('ещё', None),
 ('более', None),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', None)]

0.4067191874470994

In [46]:
unigram_bigram_tagger = UnigramTagger(fdata_train, backoff=bigram_tagger)
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', None),
 ('к', 'ADP'),
 ('ещё', 'PART'),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.7160166677518067

In [49]:
bigram_unigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'PART'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.8839768214076438

Попробуем комбинацию тэггеров

In [83]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

0.843317053722715

Для данной задачи максимальную оценку показала комбинация bigram + unigram tagger

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import catboost

In [10]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [11]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [12]:
test_enc_labels = le.transform(test_label)

In [13]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'None', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB',
       'X'], dtype='<U5')

In [14]:
len(le.classes_)

18

In [28]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=500)

In [29]:
X_train = hvectorizer.fit_transform(train_tok)

In [30]:
X_test = hvectorizer.transform(test_tok)

In [31]:
X_train.shape

(1206302, 500)

In [32]:
lr = LogisticRegression(random_state=42, max_iter=10)
lr.fit(X_train, train_enc_labels)

/Users/alexey/opt/anaconda3/envs/ml_3_9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10, random_state=42)

In [33]:
pred = lr.predict(X_test)

In [34]:
accuracy_score(test_enc_labels, pred)

0.6093105019858064

Как видимо наибольшая точность получилась при использовании тэггера bigram_unigram_tagger